# Домашнее задание 3
### Выполнила: Кудрявцева Полина Дмитриевна

In [49]:
# импорт библиотек
import pandas as pd
import sqlite3
from datetime import datetime as dt

## Задача 1

**Необходимо скачать CSV-файл - «transactions_for_dz2.csv», создать таблицу transaction_bd со всеми полями, загрузить данные из файла в таблицу.**

**Также необходимо скачать второй CSV-файл - «client_info.csv», создать таблицу customer_bd со всеми полями, загрузить данные из файла в таблицу**



In [50]:
# Прочитаем данные из файлов и посмотрим на них
transactions_df = pd.read_csv('./data/transactions_for_dz2.csv')
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66


In [51]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   TRANSACTION_ID  1048575 non-null  int64  
 1   TX_DATETIME     1048575 non-null  object 
 2   CUSTOMER_ID     1048575 non-null  int64  
 3   TERMINAL_ID     1048575 non-null  int64  
 4   TX_AMOUNT       1048575 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 40.0+ MB


In [52]:
# Прочитаем данные из файлов и посмотрим на них
clients_df = pd.read_csv('./data/client_info.csv', sep=';')
clients_df.head()

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148
2,2015-07-18,2021-01-01,Charlotte,1979,2293
3,2015-07-19,2021-01-01,Amelia,1979,1867
4,2015-07-20,2021-01-01,Ava,1979,1767


In [53]:
clients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4988 entries, 0 to 4987
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   START_DT     4988 non-null   object
 1   END_DT       4988 non-null   object
 2   CLIENT_NAME  4988 non-null   object
 3   YEAR_BIRTH   4988 non-null   int64 
 4   CLIENT_ID    4988 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 195.0+ KB


In [54]:
# Подключимся (и при отсутствии создадим) к базе данных Transactions.db
con = sqlite3.connect("./data/Transactions.db")

In [55]:
# Инициируем курсор
cur = con.cursor()

In [56]:
# Напишем SQL-запрос для создания таблиц transaction_bd и customer_bd со всеми полями из CSV файлов и выполним его
create_transactuins = '''
CREATE TABLE transaction_bd(
    TRANSACTION_ID integer NOT NULL PRIMARY KEY,
    TX_DATETIME datetime NOT NULL,
    CUSTOMER_ID integer NOT NULL,
    TERMINAL_ID integer NOT NULL,
    TX_AMOUNT real NOT NULL
);
'''
create_customers = '''
CREATE TABLE customer_bd(
    CLIENT_ID integer NOT NULL PRIMARY KEY,
    CLIENT_NAME text NOT NULL,
    YEAR_BIRTH integer NOT NULL,
    START_DT datetime NOT NULL,
    END_DT datetime NOT NULL
);
'''

cur.execute(create_transactuins)
cur.execute(create_customers)
con.commit()

In [57]:
# Запишем данные из датафреймов в таблицы
transactions_df.to_sql("transaction_bd", con, if_exists="replace", index=False)
clients_df.to_sql("customer_bd", con, if_exists="replace", index=False)

4988

## Задача 2 - SQL-запросы к БД
**a. Найти имя клиента/клиентов с максимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).**

In [58]:
current_dt = dt.strftime(dt.now(), '%Y-%m-%d')
# Напишем SQL-запрос
query_a = f'''
WITH cte AS (
SELECT t.CUSTOMER_ID, c.CLIENT_NAME, SUM(t.TX_AMOUNT) AS TOTAL_TX_AMOUNT
FROM transaction_bd t
JOIN customer_bd c ON c.CLIENT_ID = t.CUSTOMER_ID
WHERE c.END_DT >= '{current_dt}'
GROUP BY t.CUSTOMER_ID
)
SELECT CUSTOMER_ID, CLIENT_NAME, TOTAL_TX_AMOUNT
FROM cte
WHERE TOTAL_TX_AMOUNT = (SELECT MAX(TOTAL_TX_AMOUNT) FROM cte)
;
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_a = pd.read_sql(query_a, con)
df_a.head(10)

,CUSTOMER_ID,CLIENT_NAME,TOTAL_TX_AMOUNT
0,2891,Kylee,786115.87


**b. Найти имя клиента/клиентов с минимальной суммой транзакций за весь период (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).** 

In [59]:
# Напишем SQL-запрос
query_b = f'''
WITH cte AS (
SELECT t.CUSTOMER_ID, c.CLIENT_NAME, SUM(t.TX_AMOUNT) AS TOTAL_TX_AMOUNT
FROM transaction_bd t
JOIN customer_bd c ON c.CLIENT_ID = t.CUSTOMER_ID
WHERE c.END_DT >= '{current_dt}'
GROUP BY t.CUSTOMER_ID
)
SELECT CUSTOMER_ID, CLIENT_NAME, TOTAL_TX_AMOUNT
FROM cte
WHERE TOTAL_TX_AMOUNT = (SELECT MIN(TOTAL_TX_AMOUNT) FROM cte)
;
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_b = pd.read_sql(query_b, con)
df_b.head(10)

,CUSTOMER_ID,CLIENT_NAME,TOTAL_TX_AMOUNT
0,486,Kloe,30.48


**c. Найти сумму транзакций относительно даты 2023-04-01 для клиентов, у которых id начинается с 111 (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Транзакции учитываются только после завершении дня.**

In [60]:
# Напишем SQL-запрос
query_c = '''
SELECT t.CUSTOMER_ID, SUM(t.TX_AMOUNT) AS TOTAL_TX_AMOUNT
FROM transaction_bd t
JOIN customer_bd c ON c.CLIENT_ID = t.CUSTOMER_ID
WHERE c.END_DT >= '2023-04-01'
    AND SUBSTRING(CAST(t.CUSTOMER_ID as varchar), 1, 3) = '111'
    AND t.TX_DATETIME < '2023-04-01'
GROUP BY t.CUSTOMER_ID
ORDER BY TOTAL_TX_AMOUNT DESC;
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_c = pd.read_sql(query_c, con)
df_c.head(10)

,CUSTOMER_ID,TOTAL_TX_AMOUNT
0,1118,353003.39
1,1113,229558.85
2,1119,221192.71
3,1116,191203.59
4,1114,108622.52
5,1117,52751.58
6,1115,38690.76
7,1112,35273.16


**d. Найти сумму транзакций относительно года рождения клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию года рождения**

In [61]:
# Напишем SQL-запрос
query_c = f'''
SELECT c.YEAR_BIRTH, SUM(t.TX_AMOUNT) AS TOTAL_TX_AMOUNT
FROM transaction_bd t
JOIN customer_bd c ON c.CLIENT_ID = t.CUSTOMER_ID
WHERE c.END_DT >= '{current_dt}'
GROUP BY c.YEAR_BIRTH
ORDER BY YEAR_BIRTH DESC;
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_c = pd.read_sql(query_c, con)
df_c.head(10)

,YEAR_BIRTH,TOTAL_TX_AMOUNT
0,2000,8.780862e+07
1,1999,1.041507e+06
2,1998,4.348492e+06
3,1997,8.479943e+06
4,1996,6.457570e+06
5,1995,6.321200e+06
6,1994,6.244281e+06
7,1993,5.732058e+06
8,1992,5.251433e+06
9,1991,7.141074e+06


**e. Найти количество транзакций для каждого клиента (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся). Сортировать по убыванию количества транзакций.**

In [62]:
# Напишем SQL-запрос
query_e = f'''
SELECT t.CUSTOMER_ID, COUNT(DISTINCT t.TRANSACTION_ID) AS TRANSACTION_CNT
FROM transaction_bd t
JOIN customer_bd c ON c.CLIENT_ID = t.CUSTOMER_ID
WHERE c.END_DT >= '{current_dt}'
GROUP BY t.CUSTOMER_ID
ORDER BY TRANSACTION_CNT DESC
;
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_e = pd.read_sql(query_e, con)
df_e.head(10)

,CUSTOMER_ID,TRANSACTION_CNT
0,2891,477
1,2580,464
2,1953,455
3,2932,454
4,3864,453
5,1246,453
6,2470,452
7,4231,451
8,1906,451
9,379,451


## Задача 3
**Найти сумму транзакций за каждый месяц (январь, февраль, март, апрель) для всех клиентов (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).**

In [63]:
# Напишем SQL-запрос
query_f = f'''
SELECT strftime('%Y-%m', t.TX_DATETIME) as "month", SUM(t.TX_AMOUNT) AS TOTAL_TX_AMOUNT
FROM transaction_bd t
JOIN customer_bd c ON c.CLIENT_ID = t.CUSTOMER_ID
WHERE c.END_DT >= '{current_dt}'
GROUP BY  "month"
ORDER BY  "month";
'''

# Выгрузим результаты запроса в датафрейм и выведем первые 10 строк
df_f = pd.read_sql(query_f, con)
df_f.head(10)

,month,TOTAL_TX_AMOUNT
0,2023-01,1.275552e+08
1,2023-02,1.179362e+08
2,2023-03,1.273077e+08
3,2023-04,8.018857e+07


In [64]:
# Закроем подключение к БД
con.close()